In [2]:
#!/usr/bin/env python3
"""
train_arcface_finetune_true.py
True ArcFace fine-tuning with ResNet50 backbone,
Top-1 and Top-5 metrics, and plotted training curves.
"""
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import json
import tensorflow_addons as tfa



# ==============================
# USER CONFIG
# ==============================
OUTPUT_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\arcface_RAdam"
os.makedirs(OUTPUT_DIR, exist_ok=True)
train_dir  = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1\train"
val_dir    = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1\val"


PK_P, PK_K = 16, 4
IMG_SIZE   = (224, 224)
BATCH_SIZE = PK_P * PK_K

# ==============================
# Dataset
# ==============================
train_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.03),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.1)
])


def preprocess_rgb(x, y):
    # convert grayscale -> RGB if needed
    x = tf.image.grayscale_to_rgb(x) if x.shape[-1] == 1 else x
    return x, y

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True,
    color_mode='rgb'  # <-- ensure it's loaded as 3-channel
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False,
    color_mode='rgb'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")


AUTOTUNE = tf.data.AUTOTUNE
train_ds = (
    train_ds
    .map(preprocess_rgb)
    .map(lambda x, y: (train_augment(x, training=True), y))
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)   # ✅ at the END
)

val_ds = (
    val_ds
    .map(preprocess_rgb)
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)   # ✅ at the END
)


# --- compute all labels from the training dataset ---
all_labels = np.array([label.numpy() for (x, y), label in train_ds.unbatch()])


# --- compute class weights automatically ---
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=all_labels
)

# --- convert to dictionary for Keras ---
class_weights = dict(enumerate(class_weights))


# ==============================
# 🧠 Custom ArcFace Layer
# ==============================
class ArcFace(tf.keras.layers.Layer):
    def __init__(self, num_classes, s=30.0, m=0.35, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.s = s
        self.m = m

    def build(self, input_shape):
        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[-1]), self.num_classes),
            initializer='glorot_uniform',
            trainable=True
        )

    def call(self, inputs, labels):
        # unchanged
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        cos_theta = tf.matmul(x, W)
        cos_theta = tf.clip_by_value(cos_theta, -1.0, 1.0)
        theta = tf.acos(cos_theta)
        selected = tf.gather_nd(theta, tf.stack([tf.range(tf.shape(labels)[0]), labels], axis=1))
        theta_y_m = selected + self.m
        mask = tf.one_hot(labels, depth=self.num_classes)
        cos_theta_m = tf.cos(theta_y_m)

        cos_theta_m_expanded = tf.expand_dims(cos_theta_m, axis=1)
        logits = cos_theta * (1 - mask) + cos_theta_m_expanded * mask
        logits *= self.s
        return logits

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": int(self.num_classes),
            "s": float(self.s),
            "m": float(self.m)
        })
        return config


# ==============================
# Build True ArcFace Model
# ==============================
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet',
    pooling='avg', input_shape=IMG_SIZE + (3,))
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
labels = tf.keras.Input(shape=(), dtype=tf.int32)
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(512, activation=None,kernel_regularizer=tf.keras.regularizers.l2(1e-4), name="embedding" )(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)
logits = ArcFace(num_classes)(x, labels)
model = tf.keras.Model(inputs=[inputs, labels], outputs=logits, name="arcface_resnet50")

# ==============================
# Callbacks
# ==============================
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, "1_RAdam_best_arcface_weights.ckpt"),
        monitor="val_top1_acc",
        save_best_only=True,
        save_weights_only=True,   # <<--- important
        mode="max",
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_top1_acc", factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_top1_acc", patience=6, restore_best_weights=True, verbose=1)
]



# ==============================================
# 🧱 STAGE 1 — Freeze backbone (stabilize ArcFace head)
# ==============================================

initial_lr = 2e-4
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=initial_lr),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)


# ==============================
# Train
# ==============================
EPOCHS_1 = 15
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_1, callbacks=callbacks
    ,class_weight=class_weights
)



# ==============================================
# 🔓 STAGE 2 — Unfreeze backbone (fine-tune entire model)
# ==============================================

print("\n🔓 Stage 2: Unfreezing backbone and fine-tuning entire model...")
N_UNFREEZE = 50  # try 30–70 depending on dataset size

# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze last N layers (but skip BN layers)
for layer in base_model.layers[-N_UNFREEZE:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)

# 🔒 Keep all BatchNorm layers frozen globally
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False


model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)

EPOCHS_2 = 25
total_epochs = EPOCHS_1 + EPOCHS_2

history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=EPOCHS_1,
    callbacks=callbacks ,class_weight=class_weights
)

# Combine histories for full plot
def merge_histories(h1, h2):
    # handle both Keras History objects and dicts
    h1 = h1.history if hasattr(h1, "history") else h1
    h2 = h2.history if hasattr(h2, "history") else h2

    hist = {}
    for k in h1.keys():
        hist[k] = h1[k] + h2[k]
    return hist

full_history = merge_histories(history1, history2)



# ==============================
# Plot training graphs
# ==============================
def plot_history(history_dict, out_dir):
    epochs = np.arange(1, len(history_dict['loss']) + 1)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, history_dict['loss'], label='train_loss')
    plt.plot(epochs, history_dict['val_loss'], label='val_loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.subplot(1,2,2)
    plt.plot(epochs, history_dict['top1_acc'], label='train_top1')
    plt.plot(epochs, history_dict['val_top1_acc'], label='val_top1')
    plt.plot(epochs, history_dict['top5_acc'], label='train_top5', linestyle='--')
    plt.plot(epochs, history_dict['val_top5_acc'], label='val_top5', linestyle='--')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
    plt.tight_layout()
    plot_path = os.path.join(out_dir, "1_RAdam_training_history.png")
    plt.savefig(plot_path, dpi=200)
    plt.close()
    print(f"[save] training plot: {plot_path}")

plot_history(full_history, OUTPUT_DIR)

# Save final history to JSON
history_path = os.path.join(OUTPUT_DIR, "1_RAdam_full_history.json")
with open(history_path, "w") as f:
    json.dump(full_history, f)
print(f"[save] full training history: {history_path}")


print("\n✅ Training completed. Best model saved as '1_RAdam_best_arcface_model.keras'.")
print(f"Training history plot saved to: {os.path.join(OUTPUT_DIR, '1_RAdam_training_history.png')}")


Found 4606 files belonging to 455 classes.
Found 550 files belonging to 455 classes.
Found 455 unique classes.
Epoch 1/15
72/72 [==============================] - ETA: 0s - loss: 17.2116 - top1_acc: 0.0000e+00 - top5_acc: 0.0000e+00
Epoch 1: val_top1_acc improved from -inf to 0.00000, saving model to C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\arcface_RAdam\1_RAdam_best_arcface_weights.ckpt
72/72 [==============================] - 197s 3s/step - loss: 17.2116 - top1_acc: 0.0000e+00 - top5_acc: 0.0000e+00 - val_loss: 16.8669 - val_top1_acc: 0.0000e+00 - val_top5_acc: 0.0000e+00 - lr: 2.0000e-04
Epoch 2/15
72/72 [==============================] - ETA: 0s - loss: 16.6405 - top1_acc: 0.0000e+00 - top5_acc: 0.0000e+00
Epoch 2: val_top1_acc did not improve from 0.00000
72/72 [==============================] - 222s 3s/step - loss: 16.6405 - top1_acc: 0.0000e+00 - top5_acc: 0.0000e+00 - val_loss: 16.0966 - val_top1_acc: 0.0000e+00 - val_top5_acc: 0.0000e+00 - lr: 2.0000e-04
Epoc

KeyboardInterrupt: 

With the modification to save the comparision parameter like the model size, training time,inference time, etc

In [ ]:
#!/usr/bin/env python3
"""
Modified training script with automatic comparison tracking
Just add 3 lines to your existing code!
"""
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import time  # ← ADD THIS
import json
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import tensorflow_addons as tfa

# ==============================
# USER CONFIG
# ==============================
OUTPUT_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\arcface_RAdam"
os.makedirs(OUTPUT_DIR, exist_ok=True)
train_dir  = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1\train"
val_dir    = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_1\val"

PK_P, PK_K = 16, 4
IMG_SIZE   = (224, 224)
BATCH_SIZE = PK_P * PK_K

# ═══════════════════════════════════════════════════════════
# STEP 1: ADD THIS - Start tracking time
# ═══════════════════════════════════════════════════════════
TRAINING_START_TIME = time.time()
MODEL_NAME = "ArcFace_ResNet50_RAdam"  # Change this for each model
OPTIMIZER_NAME = "RAdam"
INITIAL_LR = 2e-4

# ==============================
# Dataset (your existing code)
# ==============================
train_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.03),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.1)
])

def preprocess_rgb(x, y):
    x = tf.image.grayscale_to_rgb(x) if x.shape[-1] == 1 else x
    return x, y

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True,
    color_mode='rgb'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False,
    color_mode='rgb'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")

AUTOTUNE = tf.data.AUTOTUNE
train_ds = (
    train_ds
    .map(preprocess_rgb)
    .map(lambda x, y: (train_augment(x, training=True), y))
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)
)

val_ds = (
    val_ds
    .map(preprocess_rgb)
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)
)

all_labels = np.array([label.numpy() for (x, y), label in train_ds.unbatch()])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=all_labels
)
class_weights = dict(enumerate(class_weights))

# ==============================
# ArcFace Layer (your existing code)
# ==============================
class ArcFace(tf.keras.layers.Layer):
    def __init__(self, num_classes, s=30.0, m=0.35, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.s = s
        self.m = m

    def build(self, input_shape):
        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[-1]), self.num_classes),
            initializer='glorot_uniform',
            trainable=True
        )

    def call(self, inputs, labels):
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        cos_theta = tf.matmul(x, W)
        cos_theta = tf.clip_by_value(cos_theta, -1.0, 1.0)
        theta = tf.acos(cos_theta)
        selected = tf.gather_nd(theta, tf.stack([tf.range(tf.shape(labels)[0]), labels], axis=1))
        theta_y_m = selected + self.m
        mask = tf.one_hot(labels, depth=self.num_classes)
        cos_theta_m = tf.cos(theta_y_m)
        # Fixed for serialization
        cos_theta_m_expanded = tf.expand_dims(cos_theta_m, axis=1)
        logits = cos_theta * (1 - mask) + cos_theta_m_expanded * mask
        logits *= self.s
        return logits
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.num_classes,
            "s": self.s,
            "m": self.m
        })
        return config

# ==============================
# Build Model (your existing code)
# ==============================
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet',
    pooling='avg', input_shape=IMG_SIZE + (3,))
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
labels = tf.keras.Input(shape=(), dtype=tf.int32)
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(512, activation=None, kernel_regularizer=tf.keras.regularizers.l2(1e-4), name="embedding")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)
logits = ArcFace(num_classes)(x, labels)
model = tf.keras.Model(inputs=[inputs, labels], outputs=logits, name="arcface_resnet50")

# ==============================
# Callbacks (your existing code)
# ==============================
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_best_model.keras"),
        monitor="val_top1_acc", save_best_only=True, mode="max", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_top1_acc", factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_top1_acc", patience=6, restore_best_weights=True, verbose=1)
]

# ==============================
# STAGE 1 - Compile & Train
# ==============================
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=INITIAL_LR),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)

EPOCHS_1 = 15
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_1, callbacks=callbacks,
    class_weight=class_weights
)

# ==============================
# STAGE 2 - Unfreeze & Fine-tune
# ==============================
print("\n🔓 Stage 2: Unfreezing backbone and fine-tuning entire model...")
N_UNFREEZE = 50

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-N_UNFREEZE:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)

for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)

EPOCHS_2 = 25
total_epochs = EPOCHS_1 + EPOCHS_2

history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=EPOCHS_1,
    callbacks=callbacks,
    class_weight=class_weights
)

# ═══════════════════════════════════════════════════════════
# STEP 2: ADD THIS - Stop tracking time
# ═══════════════════════════════════════════════════════════
TRAINING_END_TIME = time.time()

# Merge histories
def merge_histories(h1, h2):
    h1 = h1.history if hasattr(h1, "history") else h1
    h2 = h2.history if hasattr(h2, "history") else h2
    hist = {}
    for k in h1.keys():
        hist[k] = h1[k] + h2[k]
    return hist

full_history = merge_histories(history1, history2)

# ==============================
# Plot training graphs (your existing code)
# ==============================
def plot_history(history_dict, out_dir):
    epochs = np.arange(1, len(history_dict['loss']) + 1)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, history_dict['loss'], label='train_loss')
    plt.plot(epochs, history_dict['val_loss'], label='val_loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.subplot(1,2,2)
    plt.plot(epochs, history_dict['top1_acc'], label='train_top1')
    plt.plot(epochs, history_dict['val_top1_acc'], label='val_top1')
    plt.plot(epochs, history_dict['top5_acc'], label='train_top5', linestyle='--')
    plt.plot(epochs, history_dict['val_top5_acc'], label='val_top5', linestyle='--')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
    plt.tight_layout()
    plot_path = os.path.join(out_dir, f"{MODEL_NAME}_training_history.png")
    plt.savefig(plot_path, dpi=200)
    plt.close()
    print(f"[save] training plot: {plot_path}")

plot_history(full_history, OUTPUT_DIR)

# Save full history
history_path = os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_full_history.json")
with open(history_path, "w") as f:
    json.dump(full_history, f)
print(f"[save] full training history: {history_path}")


# ═══════════════════════════════════════════════════════════
# STEP 3: ADD THIS - Save all comparison metrics automatically
# ═══════════════════════════════════════════════════════════

def save_model_comparison_metrics(model, model_name, model_path, history_dict, 
                                   start_time, end_time, optimizer_name, 
                                   learning_rate, batch_size, total_epochs,
                                   output_dir):
    """
    Automatically save all comparison metrics to JSON
    """
    print("\n" + "="*70)
    print("📊 SAVING MODEL COMPARISON METRICS")
    print("="*70)
    
    metrics = {
        'model_name': model_name,
        'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
        'configuration': {
            'optimizer': optimizer_name,
            'initial_learning_rate': learning_rate,
            'batch_size': batch_size,
            'total_epochs': total_epochs,
            'architecture': 'ResNet50 + ArcFace',
            'image_size': IMG_SIZE,
            'num_classes': num_classes
        }
    }
    
    # 1. MODEL SIZE
    if os.path.exists(model_path):
        size_bytes = os.path.getsize(model_path)
        size_mb = size_bytes / (1024 * 1024)
        metrics['model_size'] = {
            'file_size_mb': round(size_mb, 2),
            'file_size_bytes': size_bytes
        }
        print(f"✓ Model Size: {size_mb:.2f} MB")
    
    # 2. PARAMETERS
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    metrics['parameters'] = {
        'total': int(total_params),
        'trainable': int(trainable_params),
        'non_trainable': int(total_params - trainable_params),
        'total_millions': round(total_params / 1e6, 2),
        'trainable_millions': round(trainable_params / 1e6, 2)
    }
    print(f"✓ Parameters: {total_params/1e6:.2f}M (Trainable: {trainable_params/1e6:.2f}M)")
    
    # 3. TRAINING TIME
    duration_seconds = end_time - start_time
    duration_minutes = duration_seconds / 60
    duration_hours = duration_minutes / 60
    hours = int(duration_hours)
    minutes = int(duration_minutes % 60)
    seconds = int(duration_seconds % 60)
    
    metrics['training_time'] = {
        'total_seconds': round(duration_seconds, 2),
        'total_minutes': round(duration_minutes, 2),
        'total_hours': round(duration_hours, 2),
        'formatted': f"{hours}h {minutes}m {seconds}s",
        'seconds_per_epoch': round(duration_seconds / total_epochs, 2)
    }
    print(f"✓ Training Time: {hours}h {minutes}m {seconds}s")
    print(f"  Average per epoch: {duration_seconds / total_epochs:.2f} seconds")
    
    # 4. PERFORMANCE METRICS
    best_val_top1 = max(history_dict['val_top1_acc'])
    best_val_top1_epoch = history_dict['val_top1_acc'].index(best_val_top1) + 1
    best_val_top5 = max(history_dict['val_top5_acc'])
    best_val_loss = min(history_dict['val_loss'])
    final_train_top1 = history_dict['top1_acc'][-1]
    
    metrics['performance'] = {
        'best_val_top1_accuracy': round(best_val_top1, 4),
        'best_val_top1_epoch': best_val_top1_epoch,
        'best_val_top5_accuracy': round(best_val_top5, 4),
        'best_val_loss': round(best_val_loss, 4),
        'final_train_top1_accuracy': round(final_train_top1, 4),
        'overfitting_gap': round(final_train_top1 - best_val_top1, 4)
    }
    print(f"✓ Best Val Top-1 Accuracy: {best_val_top1:.4f} (epoch {best_val_top1_epoch})")
    print(f"✓ Best Val Top-5 Accuracy: {best_val_top5:.4f}")
    
    # 5. TRAINING STABILITY
    if len(history_dict['val_top1_acc']) >= 5:
        last_5_acc = history_dict['val_top1_acc'][-5:]
        stability_std = np.std(last_5_acc)
        metrics['performance']['training_stability_std'] = round(stability_std, 4)
        print(f"✓ Training Stability (std of last 5 epochs): {stability_std:.4f}")
    
    # 6. INFERENCE SPEED
    print("⏱️  Measuring inference speed...")
    dummy_input = tf.random.normal((1, IMG_SIZE[0], IMG_SIZE[1], 3))
    dummy_label = tf.constant([0])
    
    # Warmup
    for _ in range(10):
        _ = model.predict([dummy_input, dummy_label], verbose=0)
    
    # Measure
    num_runs = 100
    start = time.time()
    for _ in range(num_runs):
        _ = model.predict([dummy_input, dummy_label], verbose=0)
    end = time.time()
    
    avg_time = (end - start) / num_runs
    fps = 1 / avg_time
    
    metrics['inference'] = {
        'avg_time_per_image_ms': round(avg_time * 1000, 2),
        'avg_time_per_image_seconds': round(avg_time, 4),
        'fps': round(fps, 2)
    }
    print(f"✓ Inference Speed: {avg_time * 1000:.2f} ms/image ({fps:.2f} FPS)")
    
    # 7. SAVE TO JSON
    metrics_path = os.path.join(output_dir, f"{model_name}_comparison_metrics.json")
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    
    print(f"\n✅ Saved comparison metrics to: {metrics_path}")
    print("="*70 + "\n")
    
    return metrics

# Call the function to save all metrics
comparison_metrics = save_model_comparison_metrics(
    model=model,
    model_name=MODEL_NAME,
    model_path=os.path.join(OUTPUT_DIR, f"{MODEL_NAME}_best_model.keras"),
    history_dict=full_history,
    start_time=TRAINING_START_TIME,
    end_time=TRAINING_END_TIME,
    optimizer_name=OPTIMIZER_NAME,
    learning_rate=INITIAL_LR,
    batch_size=BATCH_SIZE,
    total_epochs=total_epochs,
    output_dir=OUTPUT_DIR
)

print(f"\n✅ Training completed for {MODEL_NAME}")
print(f"   Model saved: {MODEL_NAME}_best_model.keras")
print(f"   Metrics saved: {MODEL_NAME}_comparison_metrics.json")
print(f"   Training plot: {MODEL_NAME}_training_history.png")

In [ ]:
#!/usr/bin/env python3
"""
train_arcface_finetune_true.py
True ArcFace fine-tuning with ResNet50 backbone,
Top-1 and Top-5 metrics, and plotted training curves.
"""
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import json
import tensorflow_addons as tfa



# ==============================
# USER CONFIG
# ==============================
OUTPUT_DIR = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\arcface_RAdam"
os.makedirs(OUTPUT_DIR, exist_ok=True)
train_dir  = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2\train"
val_dir    = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\DL\preprocess_dataset_2\val"


PK_P, PK_K = 16, 4
IMG_SIZE   = (224, 224)
BATCH_SIZE = PK_P * PK_K

# ==============================
# Dataset
# ==============================
train_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.03),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.1)
])


def preprocess_rgb(x, y):
    # convert grayscale -> RGB if needed
    x = tf.image.grayscale_to_rgb(x) if x.shape[-1] == 1 else x
    return x, y

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True,
    color_mode='rgb'  # <-- ensure it's loaded as 3-channel
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir, labels='inferred', label_mode='int',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False,
    color_mode='rgb'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Found {num_classes} unique classes.")


AUTOTUNE = tf.data.AUTOTUNE
train_ds = (
    train_ds
    .map(preprocess_rgb)
    .map(lambda x, y: (train_augment(x, training=True), y))
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)   # ✅ at the END
)

val_ds = (
    val_ds
    .map(preprocess_rgb)
    .map(lambda x, y: ((x, y), y))
    .prefetch(AUTOTUNE)   # ✅ at the END
)


# --- compute all labels from the training dataset ---
all_labels = np.array([label.numpy() for (x, y), label in train_ds.unbatch()])


# --- compute class weights automatically ---
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=all_labels
)

# --- convert to dictionary for Keras ---
class_weights = dict(enumerate(class_weights))


# ==============================
# 🧠 Custom ArcFace Layer
# ==============================
class ArcFace(tf.keras.layers.Layer):
    def __init__(self, num_classes, s=30.0, m=0.35, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.s = s
        self.m = m

    def build(self, input_shape):
        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[-1]), self.num_classes),
            initializer='glorot_uniform',
            trainable=True
        )

    def call(self, inputs, labels):
        # unchanged
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        cos_theta = tf.matmul(x, W)
        cos_theta = tf.clip_by_value(cos_theta, -1.0, 1.0)
        theta = tf.acos(cos_theta)
        selected = tf.gather_nd(theta, tf.stack([tf.range(tf.shape(labels)[0]), labels], axis=1))
        theta_y_m = selected + self.m
        mask = tf.one_hot(labels, depth=self.num_classes)
        cos_theta_m = tf.cos(theta_y_m)

        cos_theta_m_expanded = tf.expand_dims(cos_theta_m, axis=1)
        logits = cos_theta * (1 - mask) + cos_theta_m_expanded * mask
        logits *= self.s
        return logits

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": int(self.num_classes),
            "s": float(self.s),
            "m": float(self.m)
        })
        return config


# ==============================
# Build True ArcFace Model
# ==============================
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet',
    pooling='avg', input_shape=IMG_SIZE + (3,))
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
labels = tf.keras.Input(shape=(), dtype=tf.int32)
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(512, activation=None,kernel_regularizer=tf.keras.regularizers.l2(1e-4), name="embedding" )(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)
logits = ArcFace(num_classes)(x, labels)
model = tf.keras.Model(inputs=[inputs, labels], outputs=logits, name="arcface_resnet50")

# ==============================
# Callbacks
# ==============================
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, "2_RAdam_best_arcface_weights.ckpt"),
        monitor="val_top1_acc",
        save_best_only=True,
        save_weights_only=True,   # <<--- important
        mode="max",
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_top1_acc", factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_top1_acc", patience=6, restore_best_weights=True, verbose=1)
]



# ==============================================
# 🧱 STAGE 1 — Freeze backbone (stabilize ArcFace head)
# ==============================================

initial_lr = 2e-4
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=initial_lr),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)


# ==============================
# Train
# ==============================
EPOCHS_1 = 15
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_1, callbacks=callbacks
    ,class_weight=class_weights
)



# ==============================================
# 🔓 STAGE 2 — Unfreeze backbone (fine-tune entire model)
# ==============================================

print("\n🔓 Stage 2: Unfreezing backbone and fine-tuning entire model...")
N_UNFREEZE = 50  # try 30–70 depending on dataset size

# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze last N layers (but skip BN layers)
for layer in base_model.layers[-N_UNFREEZE:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)

# 🔒 Keep all BatchNorm layers frozen globally
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False


model.compile(
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_acc"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc")
    ]
)

EPOCHS_2 = 25
total_epochs = EPOCHS_1 + EPOCHS_2

history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=EPOCHS_1,
    callbacks=callbacks ,class_weight=class_weights
)

# Combine histories for full plot
def merge_histories(h1, h2):
    # handle both Keras History objects and dicts
    h1 = h1.history if hasattr(h1, "history") else h1
    h2 = h2.history if hasattr(h2, "history") else h2

    hist = {}
    for k in h1.keys():
        hist[k] = h1[k] + h2[k]
    return hist

full_history = merge_histories(history1, history2)



# ==============================
# Plot training graphs
# ==============================
def plot_history(history_dict, out_dir):
    epochs = np.arange(1, len(history_dict['loss']) + 1)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, history_dict['loss'], label='train_loss')
    plt.plot(epochs, history_dict['val_loss'], label='val_loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.subplot(1,2,2)
    plt.plot(epochs, history_dict['top1_acc'], label='train_top1')
    plt.plot(epochs, history_dict['val_top1_acc'], label='val_top1')
    plt.plot(epochs, history_dict['top5_acc'], label='train_top5', linestyle='--')
    plt.plot(epochs, history_dict['val_top5_acc'], label='val_top5', linestyle='--')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
    plt.tight_layout()
    plot_path = os.path.join(out_dir, "2_RAdam_training_history.png")
    plt.savefig(plot_path, dpi=200)
    plt.close()
    print(f"[save] training plot: {plot_path}")

plot_history(full_history, OUTPUT_DIR)

# Save final history to JSON
history_path = os.path.join(OUTPUT_DIR, "2_RAdam_full_history.json")
with open(history_path, "w") as f:
    json.dump(full_history, f)
print(f"[save] full training history: {history_path}")


print("\n✅ Training completed. Best model saved as '2_RAdam_best_arcface_model.keras'.")
print(f"Training history plot saved to: {os.path.join(OUTPUT_DIR, '2_RAdam_training_history.png')}")
